In [1]:
# Check core SDK version number
import azureml.core

print("SDK version:", azureml.core.VERSION)

SDK version: 0.1.4


In [2]:
from azureml.core.workspace import Workspace

ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

Found the config file in: /home/jayaubuntudsvm/Desktop/AKSDeploymentTutorial/Keras_Tensorflow/aml_config/config.json
jayavienna
jayavienna
eastus2
edf507a2-6235-46c5-b560-fd463ba2e771


In [3]:
# choose a name for the run history container in the workspace
run_history_name = 'train-on-local'

# project folder
project_folder = './sample_projects/train-on-local'

In [4]:
import os
from azureml.core import Project

project = Project.attach(workspace_object = ws,
                         history_name = run_history_name,
                         directory = project_folder)

print(project.project_directory, project.history.name, sep = '\n')

/home/jayaubuntudsvm/Desktop/AKSDeploymentTutorial/Keras_Tensorflow/sample_projects/train-on-local
train-on-local


In [5]:
%%writefile $project_folder/train.py

#Develop the model

import tensorflow as tf
import keras
from resnet152 import ResNet152
from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input, decode_predictions
import numpy as np
from PIL import Image
import wget

model = ResNet152(weights='imagenet')

#model.summary( )

wget.download('https://upload.wikimedia.org/wikipedia/commons/thumb/6/68/Lynx_lynx_poing.jpg/220px-Lynx_lynx_poing.jpg')

img_path = '220px-Lynx_lynx_poing.jpg'
#print(Image.open(img_path).size)
Image.open(img_path)

img = image.load_img(img_path, target_size=(224, 224))
img = image.img_to_array(img)
img = np.expand_dims(img, axis=0)
img = preprocess_input(img)

preds = model.predict(img)
print('Predicted:', decode_predictions(preds, top=3))

Overwriting ./sample_projects/train-on-local/train.py


In [6]:
#User-managed runs

from azureml.core.runconfig import RunConfiguration

# Editing a run configuration property on-fly.
run_config = RunConfiguration.load(project_object = project, run_config_name = "local")

run_config.environment.python.user_managed_dependencies = True
run_config.prepare_environment = True

# You can choose a specific Python environment by pointing to a Python path 
#run_config.environment.python.interpreter_path = '/home/ninghai/miniconda3/envs/sdk2/bin/python'

In [7]:
%%time 
from azureml.core.run import Run

run = Run.submit(project_object = project,
                 run_config = run_config,
                 script_to_run = 'train.py')

# Shows output of the run on stdout.
run.wait_for_completion(show_output = True)

RunId: train-on-local_1536091846

Streaming azureml-logs/80_driver_log.txt

/anaconda/envs/py35/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
2018-09-04 20:10:48.477611: I tensorflow/core/platform/cpu_feature_guard.cc:140] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2018-09-04 20:10:48.621588: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1356] Found device 0 with properties: 
name: Tesla K80 major: 3 minor: 7 memoryClockRate(GHz): 0.8235
pciBusID: 6f0b:00:00.0
totalMemory: 11.17GiB freeMemory: 11.10GiB
2018-09-04 20:10:48.621627: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1435] Adding visible gpu devices: 0
2018-09-04 20:10:48.872163: I tensorflow/core/com

In [8]:
# make sure you have helpers.py in the current folder
import helpers
print(helpers.get_run_history_url(run))

https://mlworkspace.azureml-test.net/portal/subscriptions/edf507a2-6235-46c5-b560-fd463ba2e771/resourceGroups/jayavienna/providers/Microsoft.MachineLearningServices/workspaces/jayavienna/history/train-on-local/runs/train-on-local_1536091846


In [10]:
# Docker based execution

from azureml.core.conda_dependencies import CondaDependencies

# Editing a run configuration property on-fly.
run_config = RunConfiguration.load(project_object = project, run_config_name = "local")

# enable Docker
run_config.environment.docker.enabled = True
print("base image to user:", run_config.environment.docker.base_image)

# Build a new conda environment from the conda_dependencies.yml file inside of the Docker container
run_config.environment.python.user_managed_dependencies = False

# Automatically create the conda environment before the run
run_config.prepare_environment = True

# add additional packages to the conda_dependencies.yml file

#cd = CondaDependencies()
#cd.add_conda_package('tensorflow==1.9.0')
#cd.add_pip_package(‘a_pip_pkg_that_cannot_be_installed_via_conda’)
#runconfig = RunConfig(conda_dependencies=cd)

cd = CondaDependencies()
cd.add_conda_package('python=3.6')
cd.add_conda_package('tensorflow')
cd.add_conda_package('tornado==4.5.3')

cd.add_pip_package('papermill==0.14.1')
cd.add_pip_package('python-dotenv==0.9.0')
cd.add_pip_package('Pillow==5.2.0')
cd.add_pip_package('wget==3.2')
cd.add_pip_package('aiohttp==3.3.2')
cd.add_pip_package('toolz==0.9.0')
cd.add_pip_package('tqdm==4.23.4')
cd.add_pip_package('azure-cli==2.0.41')
#cd.add_pip_package('tensorflow')
cd.add_pip_package('keras==2.2.0')

cd.save_to_file(project_dir = project_folder, conda_file_path = run_config.environment.python.conda_dependencies_file)

base image to user: continuumio/miniconda3:4.4.10


'aml_config/conda_dependencies.yml'

In [11]:
%%time 
print("Submitting an experiment.")
run = Run.submit(project_object = project,
                 run_config = run_config, 
                 script_to_run = "train.py")

# Shows output of the run on stdout.
run.wait_for_completion(show_output = True)

Submitting an experiment.
RunId: train-on-local_1536091959

Streaming azureml-logs/60_control_log.txt

Uploading experiment status to history service.
Adding run profile attachment azureml-logs/60_control_log.txt
Logging experiment preparation status in history service.
Running: ['sudo', 'docker', 'build', '-f', 'azureml-setup/Dockerfile', '-t', 'azureml/azureml_bce24304934b364a91b9452e2beac9e2', '.']
Sending build context to Docker daemon  205.8kB
Step 1/13 : FROM continuumio/miniconda3:4.4.10
 ---> bfb745918f77
Step 2/13 : USER root
 ---> Using cache
 ---> 3a0dd622d249
Step 3/13 : RUN mkdir -p $HOME/.cache
 ---> Using cache
 ---> bd9dfd8ddbe8
Step 4/13 : WORKDIR /
 ---> Using cache
 ---> c7d6c938ec39
Step 5/13 : COPY azureml-setup/99brokenproxy /etc/apt/apt.conf.d/
 ---> Using cache
 ---> e11a753a8158
Step 6/13 : RUN if dpkg --compare-versions `conda --version | grep -oE '[^ ]+$'` lt 4.4.0; then conda install conda==4.4.11 -c anaconda; fi
 ---> Using cache
 ---> 4707ded2ba8f
Step 7/1

  Running setup.py bdist_wheel for wget: started
  Running setup.py bdist_wheel for wget: finished with status 'done'
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
  Running setup.py bdist_wheel for toolz: started
  Running setup.py bdist_wheel for toolz: finished with status 'done'
  Stored in directory: /root/.cache/pip/wheels/f4/0c/f6/ce6b2d1aa459ee97cc3c0f82236302bd62d89c86c700219463
  Running setup.py bdist_wheel for pyyaml: started
  Running setup.py bdist_wheel for pyyaml: finished with status 'done'
  Stored in directory: /root/.cache/pip/wheels/ad/da/0c/74eb680767247273e2cf2723482cb9c924fe70af57c334513f
  Running setup.py bdist_wheel for future: started
  Running setup.py bdist_wheel for future: finished with status 'done'
  Stored in directory: /root/.cache/pip/wheels/bf/c9/a3/c538d90ef17cf7823fa51fc701a7a7a910a80f6a405bf15b1a
  Running setup.py bdist_wheel for idna-ssl: started
  Running setup.py bdist_wheel for id

mkl-random 1.0.1 requires cython, which is not installed.
mkl-fft 1.0.4 requires cython, which is not installed.
tensorflow 1.9.0 has requirement setuptools<=39.1.0, but you'll have setuptools 40.2.0 which is incompatible.
azure-cli-storage 2.2.1 has requirement azure-mgmt-storage==2.0.0rc4, but you'll have azure-mgmt-storage 2.0.0 which is incompatible.
azure-cli-acr 2.1.4 has requirement azure-mgmt-storage==2.0.0rc4, but you'll have azure-mgmt-storage 2.0.0 which is incompatible.
azure-cli-acr 2.1.4 has requirement azure-storage-blob==1.1.0, but you'll have azure-storage-blob 1.3.1 which is incompatible.
azure-mgmt-datalake-store 0.2.0 has requirement msrestazure~=0.4.7, but you'll have msrestazure 0.5.0 which is incompatible.
azure-cli-appservice 0.2.3 has requirement azure-mgmt-storage==2.0.0rc4, but you'll have azure-mgmt-storage 2.0.0 which is incompatible.
azure-mgmt-recoveryservicesbackup 0.1.1 has requirement msrestazure~=0.4.11, but you'll have msrestazure 0.5.0 which is inco


#
# To activate this environment, use:
# > source activate /azureml-envs/azureml_a517395017fe17c5d21cc41933743a63
#
# To deactivate an active environment, use:
# > source deactivate
#

Removing intermediate container a94c1a57bbb3
 ---> b6f4cb5124cf
Step 9/13 : ENV PATH /azureml-envs/azureml_a517395017fe17c5d21cc41933743a63/bin:$PATH
 ---> Running in 807b5eff6219
Removing intermediate container 807b5eff6219
 ---> 4f71da692fd8
Step 10/13 : ENV LD_LIBRARY_PATH /azureml-envs/azureml_a517395017fe17c5d21cc41933743a63/lib:$LD_LIBRARY_PATH
 ---> Running in 40b61fe6ab59
Removing intermediate container 40b61fe6ab59
 ---> 63d89c32da49
Step 11/13 : COPY azureml-setup/spark_cache.py azureml-setup/log4j.properties /azureml-setup/
 ---> d5fb2e1e7b79
Step 12/13 : RUN if [ $SPARK_HOME ]; then /bin/bash -c '$SPARK_HOME/bin/spark-submit "--repositories" "https://mmlspark.azureedge.net/maven,https://azuremldownloads.blob.core.windows.net/repo5qh91kdjs6" "--packages" "com.microsoft.ml.spark:mmlspark_2.11:

243187712/243179624 [==============================] - 2s 0us/step

40960/35363 [==================================] - 0s 0us/step
Predicted: [[('n02127052', 'lynx', 0.99591225), ('n02128385', 'leopard', 0.0011504501), ('n02123159', 'tiger_cat', 0.00094180217)]]
The experiment completed successfully. Starting post-processing steps.

Execution Summary
RunId: train-on-local_1536091959

CPU times: user 52.2 s, sys: 2.11 s, total: 54.3 s
Wall time: 46min 19s
